<div align="center"><a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"><img src="./assets/DLI_Header.png"></a></div>

# 为大规模推理部署模型

## 07 - 性能指标
-------

**目录**

* [简介](#introduction)
* [Prometheus](#prometheus)
* [练习](#exercise)
* [Docker Compose](#docker-compose)
* [后续步骤](#next-steps)

<a id="introduction"></a>
### 简介

在此 notebook 中，我们将查看 Triton 自动为我们收集的一些服务器性能指标。 Triton 提供了指示 GPU 和请求统计信息的 Prometheus 指标。默认情况下，这些指标可在 http://localhost:8002/metrics 中查看。这些指标只能通过访问端点获得，不会推送或发布到任何远程服务器。指标的格式为纯文本，因此可以直接查看，例如通过运行下面的 `curl` 命令。

In [ ]:
!curl triton:8002/metrics

<a id="prometheus"></a>
### Prometheus

为了获得更赏心悦目的查看格式，Triton 指标可与 [Prometheus](https://prometheus.io/) 兼容。将 JupyterLab URL 复制并粘贴到以下代码单元中，生成 Prometheus 窗口的链接。屏幕显示如下内容：

<div align="center"><img src="./assets/Prom.png"></div>

In [ ]:
import IPython
my_url = "FIXME"
prometheus_url = my_url.rsplit(".com", 1)[0] + ".com:9090/graph"
prometheus_url

通过查看 GPU 的利用率，我们可以检查是否过度使用硬件。这可以帮助我们诊断潜在的崩溃情况；或者如果 GPU 的利用率接近 100% 时还需要更多的资源，我们可以设置[报警](https://prometheus.io/docs/alerting/latest/overview/)。

单击地球图标并选择 `nv_gpu_utilization`。然后，单击"Execute"（执行）。

<div align="center"><img src="./assets/Prom_Add_Metric.png"></div>

此操作将生成如下行：

*nv_gpu_utilization{gpu_uuid="GPU-76eed5e4-a509-ea60-8ce3-5c9b82f9252b", instance="triton:8002", job="prometheus"}*

单击 `Graph` 选项卡，系统将显示过去一小时的 GPU 利用率图。

<a id="exercise"></a>
### 练习 3 - 制作您自己的控制面板

可以并排比较其它指标。向下滑动至 Prometheus 窗口底部，然后单击左侧的"Add Panel"（添加面板）按钮。我们可以借此操作重复上述流程。添加更多的您所选则的指标，并分析您先前的活动。您能回忆起在之前的 notebook 中，是何时向 Triton 发送请求的吗？

<a id="docker-compose"></a>
### Docker Compose

设置 Triton 和 Prometheus 最直接的方法之一就是使用 [Docker Compose](https://docs.docker.com/compose/) 工具。我们可以借此工具部署多个可以共享数据和其他资源的 [Docker 容器](https://www.docker.com/resources/what-container)。我们强烈建议您先了解 [Docker 基础知识](https://www.docker.com/101-tutorial)，然后再继续学习本节内容。

现在，我们来重点了解一下 `triton`，并逐个讲解各个配置键：
* [command](https://docs.docker.com/compose/compose-file/compose-file-v2/#command)：容器构建后的运行命令。在这种情况下，如果已在本地安装了 Triton 推理服务器库，我们将运行命令来初始化服务器，具体代码[如此所述](https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/quickstart.html#run-triton-inference-server)。
*[image](https://docs.docker.com/compose/compose-file/compose-file-v2/#image)：待构建的基础映像，在这种情况下是 [Triton 推理服务器](https://catalog.ngc.nvidia.com/orgs/nvidia/containers/tritonserver/tags) 映像。
*[shm-size](https://docs.docker.com/compose/compose-file/compose-file-v2/#shm_size)：与容器共享的存储大小。在这种情况下，为提升计算效率，我们将其设置为 1GB。
*[ulimits](https://docs.docker.com/compose/compose-file/compose-file-v2/#ulimits)：每个进程打开的文件描述符的最大数量，详情请参阅此 [Stack Overflow](https://stackoverflow.com/questions/24955883/what-is-the-max-opened-files-limitation-on-linux) 帖子。
*[ports](https://docs.docker.com/compose/compose-file/compose-file-v2/#ports)：容器中的公开端口。
*[volumes](https://docs.docker.com/compose/compose-file/compose-file-v2/#volume-configuration-reference)：可在容器及其主机之间共享的目录。

下方是一个 Docker Compose 文件，该文件与本课程所使用的设置文件非常相似。

In [ ]:
import IPython
IPython.display.Code(filename="assets/docker-compose.yml", language="yaml") 

接下来，让我们重点了解以下 `prometheus` 服务。该服务比 `triton` 服务更简单，但我们仍然需要提供[配置文件](https://prometheus.io/docs/prometheus/latest/configuration/configuration/) 。以下是一些配置键。

* `global`：定义要添加到每个 Prometheus 作业的属性。
  * `scrape_interval`：作业从数据源抽取数据的频率
  * `external_labels`：要添加到任何时间序列或警报的标签
* [scrape_configs](https://prometheus.io/docs/prometheus/latest/configuration/configuration/#scrape_config)：指定目标及其交互方式。

在本例中，我们要求它每隔 5 秒就从 `triton` 指标（如上述 Docker Compose 文件中所定义）中提取一次信息。我们可以为此过程添加 `job_name`，以便更轻松地进行识别。

In [ ]:
IPython.display.Code(filename="assets/prometheus.yml", language="yaml") 

有关如何使用 Triton 推理服务器指标的其他资源，请参阅：

* [Triton 推理服务器指标文档](https://github.com/triton-inference-server/server/blob/r20.12/docs/metrics.md)
* [关于基准测试 Triton 推理服务器的 Saleforce 博文](https://blog.einstein.ai/benchmarking-tensorrt-inference-server/)


<a id="next-steps"></a>
### 后续步骤

由于此在线环境中的 Triton 推理服务器已部署就绪，因此对您的最后一个挑战是在您自己的硬件上设置好 Triton。我们希望以上资源能助您入门。如需了解更多信息，请参阅[入门指南](https://github.com/triton-inference-server/server/blob/r21.11/docs/quickstart.md)。

祝您好运！

<div align="center"><a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"><img src="./assets/DLI_Header.png"></a></div>